In [0]:
import pandas as pd
import numpy as np

In [0]:
from sklearn.metrics import f1_score
import numpy as np

def f1_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    err = 1-f1_score(y_true, np.round(y_pred))
    return 'f1_err', err

In [2]:
xtrain=pd.read_csv('xtrain.csv')
xtrain.shape

(3235, 2054)

In [7]:
y=pd.read_csv('y.csv')
xtest=pd.read_csv('xtest.csv')
y.shape

(3235, 1)

In [11]:
print(y['sentiment_class'].value_counts())

 0    1701
-1     769
 1     765
Name: sentiment_class, dtype: int64


In [0]:
df2=pd.read_csv('test.csv')

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val  = train_test_split(xtrain,y ,test_size=0.15, random_state=0)

In [0]:
from sklearn.datasets import make_classification
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

scale_pos_weight

In [185]:
model = LGBMClassifier(boosting_type='gbdt',objective='multiclass',num_class=3,metric='multi_logloss',n_estimators=665,max_depth=8,
                       num_leaves=225,learning_rate=0.3,lambda_l2=0.03,max_bin=525,use_missing=True,min_data_in_leaf=2,
                       random_seed=42,min_data_in_bin=4,bin_construct_sample_cnt=300000,is_unbalance=True,num_trees = 500,num_thread = 12,verbose=0)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print("done")

done


In [186]:
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

Accuracy: 0.475 (0.012)


Accuracy: 0.478 (0.009)

In [187]:
model.fit(X_train, y_train,eval_metric=f1_eval)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_trees` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMClassifier(bin_construct_sample_cnt=300000, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, importance_type='split',
               is_unbalance=True, lambda_l2=0.03, learning_rate=0.3,
               max_bin=525, max_depth=8, metric='multi_logloss',
               min_child_samples=20, min_child_weight=0.001, min_data_in_bin=4,
               min_data_in_leaf=2, min_split_gain=0.0, n_estimators=665,
               n_jobs=-1, num_class=3, num_leaves=225, num_thread=12,
               num_trees=500, objective='multiclass', random_seed=42,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, ...)

In [0]:

yhat = model.predict(X_val)

In [191]:
 from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_val, yhat)
cm

array([[  9,  98,  10],
       [ 42, 177,  24],
       [ 24,  85,  17]])

In [0]:
array([[ 11,  94,  12],
       [ 26, 192,  25],
       [ 15,  91,  20]])
array([[ 16,  94,   7],
       [ 23, 195,  25],
       [ 13, 100,  13]])

In [192]:
y_pred2 = model.predict(xtest)
y_pred2.shape

(1387,)

In [193]:
ID=df2['id']
ID.shape

(1387,)

In [195]:
pred=pd.DataFrame(y_pred2)
datasets=pd.concat([ID,pred], axis=1)

datasets.columns =['id', 'sentiment_class']
datasets.to_csv('lgbm32.csv',index=False)
!cp lgbm32.csv "drive/My Drive/"

print(pred[0].value_counts())

 0    1037
-1     175
 1     175
Name: 0, dtype: int64


In [0]:
import pickle 


In [0]:
lightgbm1 = pickle.dumps(model)

In [0]:
lgbm = pickle.loads(lightgbm1)

In [202]:
from sklearn.externals import joblib

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [207]:
# Save the model as a pickle in a file 
joblib.dump(model, 'lgbm.pkl') 

['lgbm.pkl']